In [59]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.info()
train.head()
train.shape, test.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Score              10000 non-null  int64  
 9   Credit_Mix                10000 non-null  object 
 10  Payment_Behaviour         10000 non-null  object 
 11  Age                       10000 non-null  float64
 12  Annual_Income             10000 non-null  float64
 13  Num_Bank_Accounts         10000 non-null  float64
 14  Num_Cre

((10000, 21), (10000, 20))

In [61]:
train.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5269,4591,3416


In [63]:
train['Credit_Score'].value_counts()

Credit_Score
2    5237
1    2978
3    1785
Name: count, dtype: int64

In [65]:
# 데이터 전처리
target = train.pop('Credit_Score')
cols = train.columns[train.dtypes == object]
cols

Index(['Payment_of_Min_Amount', 'Credit_Mix', 'Payment_Behaviour'], dtype='object')

In [67]:
train['Payment_of_Min_Amount'] = train['Payment_of_Min_Amount'].astype('category')
train['Credit_Mix'] = train['Credit_Mix'].astype('category')
train['Payment_Behaviour'] = train['Payment_Behaviour'].astype('category')

test['Payment_of_Min_Amount'] = test['Payment_of_Min_Amount'].astype('category')
test['Credit_Mix'] = test['Credit_Mix'].astype('category')
test['Payment_Behaviour'] = test['Payment_Behaviour'].astype('category')

# train.info()
train[cols].info()
train[cols].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Payment_of_Min_Amount  10000 non-null  category
 1   Credit_Mix             10000 non-null  category
 2   Payment_Behaviour      10000 non-null  category
dtypes: category(3)
memory usage: 29.9 KB


,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
0,Yes,Bad,High_spent_Medium_value_payments
1,Yes,Bad,High_spent_Small_value_payments
2,Yes,Bad,Low_spent_Large_value_payments
3,NM,Standard,High_spent_Small_value_payments
4,Yes,Bad,High_spent_Medium_value_payments


In [69]:
# 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((8000, 20), (2000, 20), (8000,), (2000,))

In [41]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(random_state=0)
# rf.fit(X_train, y_train)
# ValueError - 숫자로 인코딩하지 않아서

In [47]:
# LightGBM 분류 모델 사용
!pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.4 MB 991.0 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.4 MB 5.6 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.4 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 8.3 MB/s eta 0:00:00


In [77]:
# LightGBM 분류 모델 사용
from sklearn.metrics import accuracy_score, f1_score
import lightgbm as lgb

# 학습
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=1)
lgbmc.fit(X_train, y_train)
pred = lgbmc.predict(X_val)
pred

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003700 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2529
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 20
[LightGBM] [Info] Start training from score -1.208148
[LightGBM] [Info] Start training from score -0.643881
[LightGBM] [Info] Start training from score -1.737271


array([2, 2, 2, ..., 3, 2, 2], dtype=int64)

In [83]:
# 평가
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

f1 = f1_score(y_val, pred, average="macro")
print('f1_score:', f1)

accuracy_score: 0.6985
f1_score: 0.6777379561595467


In [87]:
# 예측 및 결과 파일 생성
pred2 = lgbmc.predict(test)
pred2

submit = pd.DataFrame({
    'pred': pred2
})
submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

,pred
0,2
1,1
2,1
3,2
4,1
...,...
9995,2
9996,2
9997,1
9998,1
